In [1]:
warning_status = "ignore" #@param ["ignore", "always", "module", "once", "default", "error"]
import warnings
warnings.filterwarnings(warning_status)
with warnings.catch_warnings():
    warnings.filterwarnings(warning_status, category=DeprecationWarning)
    warnings.filterwarnings(warning_status, category=UserWarning)

import numpy as np
import os
#@markdown This sets the styles of the plotting (default is styled like plots from [FiveThirtyeight.com](https://fivethirtyeight.com/)
matplotlib_style = 'fivethirtyeight' #@param ['fivethirtyeight', 'bmh', 'ggplot', 'seaborn', 'default', 'Solarize_Light2', 'classic', 'dark_background', 'seaborn-colorblind', 'seaborn-notebook']
import matplotlib.pyplot as plt; plt.style.use(matplotlib_style)
import matplotlib.axes as axes;
from matplotlib.patches import Ellipse
#%matplotlib inline
import seaborn as sns; sns.set_context('notebook')

import torch
import torch.nn.functional as F
import pyro
import pyro.distributions as dist
from pyro.infer import EmpiricalMarginal

In [13]:
def model():
    A_alias = ['off', 'on']
    B_alias = ['off', 'on']
    C_alias = ['off', 'on']

    A_probs = torch.tensor([0.5, 0.5])

    B_probs = torch.tensor([
            [0.8, 0.2],
            [0.1, 0.9]
    ]) 
    C_probs = torch.tensor([
            [[0.90, 0.10], [0.99, 0.01]],
            [[0.10, 0.9], [0.40, 0.60]]])


    A = pyro.sample("A", pyro.distributions.Categorical(probs=A_probs)) 
    B = pyro.sample("B",  pyro.distributions.Categorical(probs=B_probs[A])) 
    C = pyro.sample("C", pyro.distributions.Categorical(probs=C_probs[B][A]))

    return{'A': A,
           'B': B,
           'C': C
          }

In [17]:

intervened_model = pyro.condition( pyro.do( model, data={'B': torch.tensor(1) } ), data={'C': torch.tensor(1) } )
conditioned_model = pyro.condition(model,  data={"B": torch.tensor(1), "C": torch.tensor(1)})

In [18]:
posterior_intervened = pyro.infer.Importance(intervened_model, num_samples=5000).run()
marginal_intervened = EmpiricalMarginal(posterior_intervened, "A")
samples_intervened = [marginal_intervened().item() for _ in range(5000)]
unique_intervened, counts_intervened = np.unique(samples_intervened, return_counts=True)
p_a_doB = counts_intervened[1]/counts_intervened.sum()
p_a_doB

0.3954

In [22]:
p(A|do(B), C) = t_counts[1] / t_counts.sum()

array([0, 1])

In [23]:
 t_counts[1] / t_counts.sum()

0.3296

In [5]:
posterior_conditioned = pyro.infer.Importance(conditioned_model, num_samples=5000).run()
marginal_conditioned = EmpiricalMarginal(posterior_conditioned, "A")
samples_conditioned = [marginal_conditioned().item() for _ in range(5000)]
unique_conditioned, counts_conditioned = np.unique(samples_intervened, return_counts=True)
p_a_B = counts_conditioned[1]/counts_conditioned.sum()
p_a_B

0.3988

In [45]:
counts_conditioned

array([3345, 1655])

In [46]:
counts_intervened

array([3345, 1655])